In [3]:
from keras.preprocessing import sequence
from keras.preprocessing import text
from keras.preprocessing.text import Tokenizer
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding, LSTM
from keras.layers import Conv1D, Flatten, MaxPooling1D, GlobalMaxPooling1D
from keras.preprocessing import text
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard
import os
import pandas as pd
import string
from bs4 import BeautifulSoup
import re
import random
from keras.layers import Dropout
import os


Using TensorFlow backend.


In [9]:
df = pd.read_csv(r'C:\Users\Alexis\Documents\Data_Scraping\data\{}.csv'.format("twitter_sentiment_1"), encoding="Latin-1")
df['target'] = df["target"].replace(4, 1) # replace labels of 4 with 1, easier for the computer to compute
df.drop(["ids"," flag"," user"],axis=1,inplace=True) # drop unneeded columns

In [10]:
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.tokenize import word_tokenize

def cleaning(data):
    porter = PorterStemmer()
    lancaster= LancasterStemmer()
    # calibrated stop word list
    updated_stopwords = ['i', 'me', 'my', 'myself', 'we', "weve", "wev", 'our', 'ours', 'ourselves', 'you', 
                         "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 
                         'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 
                         'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 
                         'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 
                         'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 
                         'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for',
                         'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above',
                         'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',
                         'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 
                         'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'only', 'own', 'same', 'so', 'than', 
                         'too', 's', 't', 'can', 'will', 'just', 'should', "should've", 'now', 'd',
                         'll', 'm', 'o', 're', 've', 'y', 'ma', "youre", "youve", "youll", "youd", 
                         "shes", "its", "thatll", "hes", "im", "x", "well", "wel", "w", "i'm", "u", "b", "theyre", "they're",
                         "ms", "mrs", "mr", "s"]
    
    text_df = data[" text"]
    list_1 = []  
    for text in text_df:
        html_decoded = BeautifulSoup(text, "lxml")# html decoding
        html_decoded = html_decoded.get_text() # gets the text from the decoded html
        
        try: 
            html_decoded = html_decoded.decode("utf-8-sig") # decoded using utf-8-sig
            no_utf = html_decoded.replace(u"\ufffd", " ") # removes UTF-8 BOM
        except: #if nothing is found then the make the html_decoded equal no_utf without going through the utf
            no_utf = html_decoded
        no_utf = no_utf.encode('utf8').decode('utf8')
        string_no_tags = re.sub(r"@[A-Za-z0-9-_]+"," ", no_utf) # removes tags
        
        no_urls = re.sub(r"http\S+", " ", string_no_tags) # remove urls
       
        no_numbers = re.sub(r"\d+", " ", no_urls) # remove numbers 
        
        no_hashtags = re.sub(r"#[A-Za-z0-9]+"," ",no_numbers) # removes hashtags and anything following it
        no_hashtags = [word.lower() for word in no_hashtags.split()]
    
        removed_stopwords = [word for word in no_hashtags if word not in updated_stopwords]
        
        stem = ""
        for words in removed_stopwords:
            stem += "{} ".format(porter.stem(words))
            
        duplicates_deleted = re.sub('\s+', ' ', stem).strip() # all spaces are converted to a single spac
  
        # removes duplicate characters
        duplicates_deleted = ""
        check = ""
        counter = 0
        for word in removed_stopwords:
            for character in word:
                counter += 1
                if character != check:
                    check = character
                    duplicates_deleted += "{}".format(character)
                else:
                    continue 
            if len(word) == counter: 
                duplicates_deleted += "{}".format(" ")
                counter = 0
                check = ""
        duplicates_deleted = duplicates_deleted.replace(".", " ")
            
        no_punctuation = "".join([char.lower() for char in duplicates_deleted if char not in string.punctuation]) # remove punctuation
        tweets = no_punctuation.lower() # turn everything lowercase
        #print(tweets)
        tweets_1 = ""
        for words in tweets.split():
            tweets_1 += "{} ".format(porter.stem(words))
        
        tweets = re.sub('\s+', ' ', tweets_1).strip() # all spaces are converted to a single spac
        list_1.append(tweets) # add each list to tweet  
        #print("filtered & cleaned tweets: " + tweets + "\n")
        
    dict_1 = {"text": list_1}  # create dictionary for dataframe structure
    cleaned_dataframe = pd.DataFrame(dict_1) # creates the dataframe
    return cleaned_dataframe

cleaned_df = cleaning(df)

In [12]:
def combine(cleaned_df, old_df):
    targetList = []
    dateList = []
    textList = []
    for f in old_df[" date"]:
        dateList.append(f)
    for f in old_df["target"]:
        targetList.append(f)
    for f in cleaned_df["text"]:
        textList.append(f)
    
    merged_df = pd.DataFrame({"date": dateList, "target": targetList,"text": textList})
    return merged_df   

merged_df = combine(cleaned_df, df)

In [13]:
for index, row in merged_df.iterrows():
    if row["text"] == "":
        merged_df.drop(index, inplace=True)
    elif "ï¿½" in row["text"]:
        merged_df.drop(index, inplace=True)
    elif "à" in row["text"]:
        merged_df.drop(index, inplace=True)
    elif "¥" in row["text"]:
        merged_df.drop(index, inplace=True)
    elif "ð" in row["text"]:
        merged_df.drop(index, inplace=True)
    elif "ñ" in row["text"]:
        merged_df.drop(index, inplace=True)
    elif "è" in row["text"]:
        merged_df.drop(index, inplace=True)
    elif "å" in row["text"]:
        merged_df.drop(index, inplace=True)
    elif "ç" in row["text"]:
        merged_df.drop(index, inplace=True)
    elif "â" in row["text"]:
        merged_df.drop(index, inplace=True)
    elif "º¥" in row["text"]:
        merged_df.drop(index, inplace=True) 
    elif "ë" in row["text"]:
        merged_df.drop(index, inplace=True)
    elif "í" in row["text"]:
        merged_df.drop(index, inplace=True)
    elif "£" in row["text"]:
        merged_df.drop(index, inplace=True)
    elif "ø" in row["text"]:
        merged_df.drop(index, inplace=True)
    elif "§" in row["text"]:
        merged_df.drop(index, inplace=True)
    elif "ù" in row["text"]:
        merged_df.drop(index, inplace=True)
    elif "ã" in row["text"]:
        merged_df.drop(index, inplace=True)

In [14]:
print(len(merged_df))
merged_df.to_csv("updated_merged_df.csv")

30


In [16]:
X = merged_df["text"]
y = merged_df["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=4)

max_words = 30000
tokenizer = Tokenizer(num_words=max_words) 
tokenizer.fit_on_texts(X) 

X_train = tokenizer.texts_to_sequences(X_train) 
X_test = tokenizer.texts_to_sequences(X_test) 

In [17]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

X_train = sequence.pad_sequences(X_train, maxlen=280)
X_test = sequence.pad_sequences(X_test, maxlen=280)

In [ ]:
max_features = 50000 # unique vocab
maxlen = 280 # the number of words per data point
embedding_size = 32 # the dimensions that each word is converted to

#tensorboard
name = "LSTM 3 layers"
tboard_log_dir = os.path.join("logs", name)
tensorboard = TensorBoard(log_dir = tboard_log_dir)

model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen)) 
model.add(LSTM(64, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(LSTM(64, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(LSTM(64, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

fitModel = model.fit(X_train, y_train, epochs = 30, batch_size = 126, validation_data=(X_test, y_test), 
                     verbose=1, callbacks=[tensorboard]) 

print(model.summary())